In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('../model_saved/b_tag_model.h5')

In [ ]:
import hls4ml
import os
os.environ['PATH'] = '/opt/Xilinx/Vivado/2020.1/bin:' + os.environ['PATH']

In [ ]:
%%capture
int_bit = 10
i = 3
reuse = 4
precision = 'ap_fixed<{},{}>'.format((int_bit+i), int_bit)
fix_int_precision = 'ap_fixed<{},{}>'.format((16+i), 14)
dirc = 'runs/B_tagging_run/r4_{}int_{}frac'.format(int_bit, i)
#First, the baseline model
hls_config = hls4ml.utils.config_from_keras_model(model, granularity='name')

# Set the precision and reuse factor for the full model
hls_config['Model']['Precision'] = precision
hls_config['Model']['ReuseFactor'] = reuse
hls_config['Model']['Strategy'] = 'Resource'

for Layer in hls_config['LayerName'].keys():
    hls_config['LayerName'][Layer]['Precision'] = precision
    hls_config['LayerName'][Layer]['Strategy'] = 'Resource'
    hls_config['LayerName'][Layer]['ReuseFactor'] = reuse
    hls_config['LayerName'][Layer]['weight'] = precision
    hls_config['LayerName'][Layer]['scale'] = precision
    hls_config['LayerName'][Layer]['bias'] = precision
    hls_config['LayerName'][Layer]['accum_t'] = fix_int_precision
    if 'multi_head_attention' in Layer:
        hls_config['LayerName'][Layer]['table_t'] = fix_int_precision
        hls_config['LayerName'][Layer]['inv_range'] = 256
        hls_config['LayerName'][Layer]['exp_range'] = 4
        hls_config['LayerName'][Layer]['table_size'] = 1024
    if 'dense' in Layer:
        hls_config['LayerName'][Layer]['table_t'] = fix_int_precision
        hls_config['LayerName'][Layer]['inv_range'] = 256
        hls_config['LayerName'][Layer]['table_size'] = 1024
        hls_config['LayerName'][Layer]['exp_range'] = 4

cfg = hls4ml.converters.create_config(backend='Vivado')
cfg['IOType']     = 'io_parallel' # Must set this if using CNNs!
cfg['HLSConfig']  = hls_config
cfg['KerasModel'] = model
cfg['OutputDir']  = dirc
cfg['Part'] = 'xcvu13p-fhga2104-2L-e'

hls_model = hls4ml.converters.keras_to_hls(cfg)
hls_model.compile()
hls_model.build(csim=False, synth=True, vsynth=True)

INFO: [XFORM 203-602] Inlining function 'fp_struct<double>::mantissa' into 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' (/wrk/2020.1/continuous/2020_05_27_2902540/src/products/hls/hls_lib/hlsmath/include/FloatingPoint/hls_case_IEEE754.h:15) automatically.
INFO: [XFORM 203-602] Inlining function 'fp_struct<double>::expv' into 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' (/wrk/2020.1/continuous/2020_05_27_2902540/src/products/hls/hls_lib/hlsmath/include/FloatingPoint/hls_case_IEEE754.h:18) automatically.
INFO: [XFORM 203-602] Inlining function 'fp_struct<double>::__signbit' into 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' (/wrk/2020.1/continuous/2020_05_27_2902540/src/products/hls/hls_lib/hlsmath/include/FloatingPoint/hls_case_IEEE754.h:59) automatically.
INFO: [XFORM 203-602] Inlining function 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' into 'generic_cast_IEEE754<int, double>' (/wrk/2020.1/continuous/2020_05_27_2902540/src/products/hls/hls_lib/hlsmath/include/Float

INFO: [XFORM 203-602] Inlining function '__hls_fptosi_double_i32' into 'nnet::dense_latency<ap_fixed<13, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<13, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, config16_1>' (firmware/nnet_utils/nnet_dense_latency.h:55) automatically.
INFO: [XFORM 203-602] Inlining function 'nnet::cast<ap_fixed<13, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<13, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, config16_1>' into 'nnet::dense_latency<ap_fixed<13, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<13, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, config16_1>' (firmware/nnet_utils/nnet_dense_latency.h:83) automatically.
INFO: [XFORM 203-602] Inlining function 'nnet::read_stream_array<ap_fixed<13, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, 6>' into 'nnet::lin_projection<ap_fixed<13, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<13, 10, (ap_q_mode)5, (ap_o_mode)3, 0>, config16>667' (firmware/nnet_utils/nnet_multiheadattention.h:217) automatically.
INFO: [XFORM 203-602] Inlining function 'nnet::read